# Importing Yelp Dataset

Author(s): Brian Lin

This notebook is for importing the Yelp JSON data files. Assumes run from
`preprocessing/`. Importing all of the following takes ~ 6 minutes on my machine.

Database file is around 5.6 GB.

In [ ]:
import json
from pathlib import Path
import sqlite3
import pandas as pd

In [ ]:
CWD = Path.cwd()
ROOT = CWD.parent
# path to raw data directory
DATA_DIR = CWD/"raw_data"
DATA_PREFIX = "yelp_academic_dataset_"
# database
DB_PATH = ROOT/"database/YelpData.db"
OUTPUT_PATH = CWD/"processed_data"/"joined.parquet.snappy"

In [ ]:
# Auto-close / commit when used in "with" statement
# https://stackoverflow.com/questions/19522505/using-sqlite3-in-python-with-with-keyword
class SQLite():
    def __init__(self, file='sqlite.db'):
        self.file=file
    def __enter__(self):
        self.conn = sqlite3.connect(self.file)
        self.conn.row_factory = sqlite3.Row
        return self.conn.cursor()
    def __exit__(self, type, value, traceback):
        self.conn.commit()
        self.conn.close()

In [ ]:
def getFilePath(name):
    return DATA_DIR/(DATA_PREFIX + name + ".json")

# convert nested obj to json str
def objToStr(data, field):
    if data[field] is not None:
        # only bother if there is a nested obj
        data[field] = json.dumps(data[field])

def replaceEmpty(data, field, replacement=None):
    if data[field] == "":
        data[field] = replacement

# dataFunc(obj): in-place operations on json object
def importData(name, valueFields, dataFunc=None):
    with SQLite(DB_PATH) as cur:
        with open(getFilePath(name), 'r') as file:
            # start reading lines from the beginning of file
            for line in file:
                data = json.loads(line)

                if dataFunc is not None:
                    dataFunc(data)

                keyStr=", ".join(valueFields)
                fieldStr=", ".join([":" + field for field in valueFields])

                sqlStr = f'''
                INSERT INTO {name} ({keyStr})
                VALUES ({fieldStr})
                '''

                cur.execute(sqlStr, data)
    print(f"Finished importing: {name}")

def printHead(name):
    with SQLite(DB_PATH) as cur:
        res = cur.execute(f"SELECT * FROM {name} LIMIT 1").fetchone()
        print(dict(res), end='\n\n')

In [ ]:
objFields = [
    "business_id",
    "name",
    "address",
    "city",
    "state",
    "postal_code",
    "latitude",
    "longitude",
    "stars",
    "review_count",
    "is_open",
    "attributes",
    "categories",
    "hours"
]

def businessFunc(data):
    replaceEmpty(data, "categories")
    for field in ["attributes", "hours"]: objToStr(data, field)

importData("business", objFields, businessFunc)
printHead("business")

In [ ]:
objFields = [
    "user_id",
    "name",
    "review_count",
    "yelping_since",
    "friends_count",
    "useful",
    "funny",
    "cool",
    "fans",
    "elite",
    "average_stars",
    "compliment_hot",
    "compliment_more",
    "compliment_profile",
    "compliment_cute",
    "compliment_list",
    "compliment_note",
    "compliment_plain",
    "compliment_cool",
    "compliment_funny",
    "compliment_writer",
    "compliment_photos"
]

def userFunc(data):
    friendStr = data["friends"]
    # make empty strings to None so sqlite converts to Null
    replaceEmpty(data, "elite")

    # friends list can be massive, and we aren't really interested in it.
    # instead, we use its count
    if friendStr == "":
        data["friends_count"] = 0
    else:
        data["friends_count"] = len(friendStr.split(","))

importData("user", objFields, userFunc)
printHead("user")

In [ ]:
objFields = [
    "review_id",
    "user_id",
    "business_id",
    "stars",
    "date",
    "text",
    "useful",
    "funny",
    "cool"
]

importData("review", objFields)
printHead("review")

In [ ]:
conn = sqlite3.connect(DB_PATH)
joinStr = '''
SELECT
    r.id AS r_id,
    b.id AS b_id,
    u.id AS u_id,
    r.stars AS r_stars,
    r.date AS r_date,
    r.text AS r_text,
    r.useful AS r_useful,
    r.funny AS r_funny,
    r.cool AS r_cool,
    b.stars AS b_stars,
    b.review_count AS b_review_count,
    u.review_count AS u_review_count,
    u.yelping_since AS u_yelping_since,
    u.friends_count AS u_friends_count
FROM review AS r
LEFT JOIN business AS b
ON r.business_id=b.business_id
LEFT JOIN user AS u
ON r.user_id=u.user_id
WHERE b.is_open<>0 AND r.useful<>0
'''
# removes closed businesses and reviews with no useful votes
data = pd.read_sql(joinStr, conn, parse_dates=['r_date', 'u_yelping_since'])
conn.close()

In [ ]:
# convert to object dtypes to string since text data is read as 'object' dtype
# by default.
data = data.astype({"r_id": "string", "r_text": "string"}, copy=False)
print(data.dtypes)
print(f"Memory usage: {data.memory_usage(index=True).sum() / 2**20} MB")
print(f"shape: {data.shape}")

In [ ]:
# save data to disk
data.to_parquet(OUTPUT_PATH, index=False, compression='snappy')
# to read back into memory:
# pd.read_parquet(OUTPUT_PATH)